In [41]:
# Simple thermostat API using FastAPI
# importing libraries
from fastapi import FastAPI, Depends
import uvicorn
import nest_asyncio
import random
import json

app = FastAPI()


In [42]:
# Defining the thermostat class
class Thermostat:
    def __init__(self,
                 name: str,
                 temperature: int,
                 humidity: int,
                 mode: str,
                 kb_id: str,
                 ):
        self.name = name
        self.temperature = temperature
        self.humidity = humidity
        self.mode = mode
        self.kb_id = kb_id


In [43]:
# Creating a thermostat object
thermostat = Thermostat(name="Thermostat",
                        temperature=20,
                        humidity=50,
                        mode="cool",
                        kb_id="http://127.0.0.1:8000/thermostat",
                        )

In [45]:
thermostat.__dict__

{'name': 'Thermostat',
 'temperature': 20,
 'humidity': 50,
 'mode': 'cool',
 'kb_id': 'http://127.0.0.1:8000/thermostat'}

In [46]:
# Defining the API endpoints
@app.get("/")
async def root():
    return {"message": "Welcome to the thermostat API"}


@app.get("/thermostat")
async def get_thermostat():
    return thermostat.__dict__


@app.get("/thermostat/name")
async def get_name():
    return thermostat.name


@app.get("/thermostat/temperature")
async def get_temperature():
    return thermostat.temperature


@app.get("/thermostat/humidity")
async def get_humidity():
    return thermostat.humidity


@app.get("/thermostat/mode")
async def get_mode():
    return thermostat.mode


@app.get("/thermostat/kb_id")
async def get_kb_id():
    return thermostat.kb_id

@app.put("/thermostat")
async def set_thermostat(commons: Thermostat = Depends()):
    thermostat.name = commons.name
    thermostat.temperature = commons.temperature
    thermostat.humidity = commons.humidity
    thermostat.mode = commons.mode
    thermostat.kb_id = commons.kb_id

    return thermostat.__dict__

@app.put("/thermostat/temperature")
async def set_temperature(temperature: int):
    thermostat.temperature = temperature
    return thermostat.temperature


@app.put("/thermostat/humidity")
async def set_humidity(humidity: int):
    thermostat.humidity = humidity
    return thermostat.humidity


@app.put("/thermostat/mode")
async def set_mode(mode: str):
    thermostat.mode = mode
    return thermostat.mode


@app.put("/thermostat/name")
async def set_name(name: str):
    thermostat.name = name
    return thermostat.name

@app.put("/thermostat/kb_id")
async def set_kb_id(kb_id: str):
    thermostat.kb_id = kb_id
    return thermostat.kb_id


In [13]:
# register API in the knowledge engine
import httpx

# Knowledge engine REST API URL
URL = "http://127.0.0.1:8280/rest"

# Knowledge engine REST API headers
HEADERS = {
    'Content-Type': 'application/json',
    'Knowledge-Base-Id': thermostat.kb_id,
}


# Creating a smart connector
def create_smart_connector():
    # Smart connector data
    sc_data = {
        "knowledgeBaseId": "http://127.0.0.1:8000/thermostat",
        "knowledgeBaseName": "Thermostat API v0.0.1",
        "knowledgeBaseDescription": "Thermostat API v0.0.1 for the Smart Home",
        "reasonerEnabled": "false",
    }

    # Registering the smart connector
    response = httpx.post(URL + "/sc", headers=HEADERS, json=sc_data)

    print(f'URL:\n {URL}\n')
    print(f'Headers:\n {HEADERS}\n')
    print(f'Smart connector data:\n {sc_data}\n')
    print(response)
    print(f'Response text: {response.text}')


create_smart_connector()

URL:
 http://127.0.0.1:8280/rest

Headers:
 {'Content-Type': 'application/json', 'Knowledge-Base-Id': 'http://127.0.0.1:8000/thermostat'}

Smart connector data:
 {'knowledgeBaseId': 'http://127.0.0.1:8000/thermostat', 'knowledgeBaseName': 'Thermostat API v0.0.1', 'knowledgeBaseDescription': 'Thermostat API v0.0.1 for the Smart Home', 'reasonerEnabled': 'false'}

<Response [200 OK]>
Response text: 


In [14]:
# check if the smart connector is registered
def check_smart_connector():
    headers2 = {
        #'accept': 'application/json,
        'Content-Type': 'application/json',
        'Knowledge-Base-Id': "http://127.0.0.1:8000/thermostat",

    }

    response = httpx.get(URL + "/sc", headers=headers2)

    print(f'URL:\n {URL}\n')
    print(f'Headers:\n {headers2}\n')
    print(response)
    print(f'Response text:\n{response.text}')


check_smart_connector()

URL:
 http://127.0.0.1:8280/rest

Headers:
 {'Content-Type': 'application/json', 'Knowledge-Base-Id': 'http://127.0.0.1:8000/thermostat'}

<Response [200 OK]>
Response text:
[{"knowledgeBaseId":"http://127.0.0.1:8000/thermostat","knowledgeBaseName":"Thermostat API v0.0.1","knowledgeBaseDescription":"Thermostat API v0.0.1 for the Smart Home","reasonerEnabled":false}]


In [15]:
# register an ANSWER Knowledge Interaction with the smart connector
def register_answer_ki():
    headers2 = {
        'Content-Type': 'application/json',
        'Knowledge-Base-Id': "http://127.0.0.1:8000/thermostat",
    }

    answer_ki_data = {
        "knowledgeInteractionType": "AnswerKnowledgeInteraction",
        "prefixes": {"rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
                     "ex": "http://example.org/",
                     },

        "graphPattern": "?d rdf:type 'Device' . ?d ex:hasName 'Thermostat'. ?d ex:hasTemperature ?temperature. ?d ex:hasHumidity ?humidity. ?d ex:hasMode ?mode. ?d ex:hasName ?name."
    }

    response = httpx.post(URL + "/sc/ki", headers=headers2, json=answer_ki_data)

    print(f'URL:\n {URL}\n')
    print(f'Headers:\n {headers2}\n')
    print(response)
    print(f'Response text:\n{response.text}')


register_answer_ki()


URL:
 http://127.0.0.1:8280/rest

Headers:
 {'Content-Type': 'application/json', 'Knowledge-Base-Id': 'http://127.0.0.1:8000/thermostat'}

<Response [200 OK]>
Response text:
{"knowledgeInteractionId":"http://127.0.0.1:8000/thermostat/interaction/0285bc86-13d4-4835-a2b7-80f61b3d1523"}


In [16]:
# check if the Knowledge Interaction is registered
def list_all_kis():
    headers2 = {
        'Content-Type': 'application/json',
        'Knowledge-Base-Id': "http://127.0.0.1:8000/thermostat",
    }

    response = httpx.get(URL + "/sc/ki", headers=headers2)

    print(f'URL:\n {URL}\n')
    print(f'Headers:\n {headers2}\n')
    print(response)
    print(f'Response text:\n{response.text}')


list_all_kis()


URL:
 http://127.0.0.1:8280/rest

Headers:
 {'Content-Type': 'application/json', 'Knowledge-Base-Id': 'http://127.0.0.1:8000/thermostat'}

<Response [404 Not Found]>
Response text:
{"messageType":"error","message":"This knowledge base has been suspended due to inactivity. Please reregister the knowledge base and its knowledge interactions."}


In [47]:
# Running the API
if __name__ == "__main__":
    nest_asyncio.apply()
    #uvicorn.run(app, host="0.0.0.0", port=8001)
    uvicorn.run(app)

INFO:     Started server process [4964]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:50538 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:50538 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:50538 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:40288 - "GET /thermostat HTTP/1.1" 200 OK
INFO:     127.0.0.1:55934 - "PUT /thermostat?name=coolboi2000&temperature=200&humidity=200&mode=chill&kb_id=kbid HTTP/1.1" 200 OK
INFO:     127.0.0.1:35588 - "GET /thermostat/name HTTP/1.1" 200 OK
INFO:     127.0.0.1:53262 - "PUT /thermostat/name?name=newname HTTP/1.1" 200 OK
INFO:     127.0.0.1:50962 - "GET /thermostat/temperature HTTP/1.1" 200 OK
INFO:     127.0.0.1:57928 - "PUT /thermostat/temperature?temperature=100 HTTP/1.1" 200 OK
INFO:     127.0.0.1:57934 - "GET /thermostat/humidity HTTP/1.1" 200 OK
INFO:     127.0.0.1:47454 - "PUT /thermostat/humidity?humidity=200 HTTP/1.1" 200 OK
INFO:     127.0.0.1:47460 - "GET /thermostat/mode HTTP/1.1" 200 OK
INFO:     127.0.0.1:57420 - "PUT /thermostat/mode?mode=sleep HTTP/1.1" 200 OK
INFO:     127.0.0.1:57430

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [4964]
